In [1]:
import pandas as pd

In [162]:
# Creates a list of the past 3 dates
def map_date(date): # date is a string
    # date in the form 1/9/16
    
    if(not date):
        return []
    
    d = date.split('/')
    month = int(d[0])
    day = int(d[1])
    year = int(d[2])
    
    previousDates = []
    for i in range(3):
        day = day - 1
        if(day == 0):
            month = month - 1
            if(month == 0):
                month = 12
                day = 31
                year = year - 1
                if(year < 0):
                    year = 99
            else:
                if(month == 2):
                    if (year % 4 == 0):
                        day = 29
                    else:
                        day = 28
                elif(month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10):
                    day = 31
                else:
                    day = 30
    
        if(len(str(year)) == 1):
            yearTemp = '0' + str(year)
        else:
            yearTemp = year
        string = str(month) + '/' + str(day) + '/' + str(yearTemp)
        previousDates.append(string)
        
    return previousDates

In [32]:
sp500 = pd.read_csv("SP500/sp500.csv")
# Rename columns
sp500.columns = ['Date', 'sp500', 'log_sp500']
sp500 = sp500.dropna()
sp500.shape

(14276, 3)

In [152]:
# Build dataframe for regression - takes about 4 minutes

cols = ['Date','Pred_Val','Day-1','Day-2','Day-3']#,'Day-4','Day-5','Day-6','Day-7','Day-8','Day-9','Day-10']
reg_data = pd.DataFrame(columns=cols)

reg_data['Date'] = sp500['Date']
reg_data['Pred_Val'] = sp500['sp500']

for index, row in sp500.iterrows():
    dates = map_date(row['Date'])
    for i, d in enumerate(dates):
        point = sp500[sp500['Date']==d]
        if(point['sp500'].empty):
            continue
        reg_data.set_value(index, reg_data.columns[i+2], point['sp500'][point.index[0]])
            

/afs/crc.nd.edu/user/p/pbui/pub/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [154]:
reg_data.dropna().shape

(5708, 5)

In [161]:
# Linear Regression Model

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_validation import KFold

data = reg_data.dropna()

predict_val = 'Pred_Val'
x = data[reg_data.columns[2:]]
kf = KFold(x.shape[0], n_folds=5, shuffle=True)
y = data[predict_val]
for train_index, test_index in kf:
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    regr = linear_model.LinearRegression()
    regr.fit(x_train, y_train)

    y_pred = regr.predict(x_test)
    print("MSE: ", mean_squared_error(y_test, y_pred))

MSE:  85.5045816249884
MSE:  79.9341193483527
MSE:  81.9501411731735
MSE:  95.11379841282414
MSE:  99.23408430399714
